# Imports 

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib as plot 

# Data upload

In [2]:
shop_data = pd.read_csv('../raw_data/shop_sales_2021-07-01_2023-08-24.csv')

In [3]:
fb_data= pd.read_csv('../raw_data/fb_2023-08-24_AllTimeExcludingNL-Guido.csv')

In [4]:
tt_data = pd.read_excel('../raw_data/tt_Daily_01072022-30112022_DEAT.xlsx')

In [5]:
google_data=pd.read_excel('../raw_data/google_export.xlsx')

# Cleaning data

## shop_data_cleanup

In [6]:
shop_data

,day,shipping_country,orders,total_sales
0,2021-07-01,Germany,10,414.85
1,2021-07-02,Austria,2,34.99
2,2021-07-02,Germany,4,194.96
3,2021-07-03,Germany,8,349.91
4,2021-07-03,Austria,2,109.98
...,...,...,...,...
1625,2023-08-23,Netherlands,19,720.36
1626,2023-08-23,Germany,121,4517.69
1627,2023-08-24,Netherlands,15,422.01
1628,2023-08-24,Austria,11,441.39


In [7]:
# List of shipping countries to exclude
countries_to_exclude = ['Netherlands', 'Italy']

# Filter out rows with shipping countries in the exclusion list
filtered_shop_data = shop_data[~shop_data['shipping_country'].isin(countries_to_exclude)]


In [8]:
shop_data_sumed= filtered_shop_data.groupby('day', as_index=False)[['orders', 'total_sales']].sum()



In [9]:
shop_data_sumed.shape

(785, 3)

In [10]:
shop_data_sumed["day"]=pd.to_datetime(shop_data_sumed["day"])

In [11]:
shop_data_sumed=shop_data_sumed.rename(columns={'day':'Day'})

In [12]:
shop_data_sumed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Day          785 non-null    datetime64[ns]
 1   orders       785 non-null    int64         
 2   total_sales  785 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 18.5 KB


In [13]:
shop_data_sumed.isnull().sum()

Day            0
orders         0
total_sales    0
dtype: int64

In [14]:
shop_data_sumed.sort_values(by='Day')

,Day,orders,total_sales
0,2021-07-01,10,414.85
1,2021-07-02,6,229.95
2,2021-07-03,10,459.89
3,2021-07-04,17,508.36
4,2021-07-05,13,399.88
...,...,...,...
780,2023-08-20,294,13616.97
781,2023-08-21,356,15348.42
782,2023-08-22,150,6054.53
783,2023-08-23,130,4810.67


In [15]:
shop_data_cleaned=shop_data_sumed

## tt_data clean up

In [16]:
tt_data

,Date,Cost,Impression,Clicks (Destination),Reach,Total Complete Payment,Total Complete Payment Value,Currency
0,2022-07-01,320.00,122647,627,78947,2,49.98,EUR
1,2022-07-02,320.00,111337,624,72902,5,181.96,EUR
2,2022-07-03,368.29,137325,667,89738,6,244.93,EUR
3,2022-07-04,418.56,158848,885,99378,14,430.34,EUR
4,2022-07-05,420.00,178959,1011,113456,7,229.93,EUR
...,...,...,...,...,...,...,...,...
149,2022-11-27,110.00,26132,82,15680,1,49.99,EUR
150,2022-11-28,77.66,19912,73,12874,5,298.68,EUR
151,2022-11-29,49.75,19804,237,13834,2,80.48,EUR
152,2022-11-30,78.68,28702,351,20751,0,0.00,EUR


In [17]:
tt_data.drop(columns=['Currency', 'Total Complete Payment', 'Total Complete Payment Value', 'Reach'], inplace=True)

In [18]:
tt_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  154 non-null    object 
 1   Cost                  154 non-null    float64
 2   Impression            154 non-null    int64  
 3   Clicks (Destination)  154 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 4.9+ KB


In [19]:
tt_data.drop(tt_data.index[153], inplace=True) 

In [20]:
tt_data["Date"]=pd.to_datetime(tt_data["Date"])

In [21]:
tt_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  153 non-null    datetime64[ns]
 1   Cost                  153 non-null    float64       
 2   Impression            153 non-null    int64         
 3   Clicks (Destination)  153 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 4.9 KB


In [22]:
tt_data_renamed=tt_data.rename(columns={'Date':'Day', 'Cost':'tt_costs', 'Impression':'tt_impressions', 'Clicks (Destination)':'tt_clicks'})

In [23]:
tt_data_cleaned=tt_data_renamed

## fb_data clean up

In [24]:
fb_data.sort_values

<bound method DataFrame.sort_values of             Day  Amount spent (EUR)  Impressions   Reach  Link clicks  \
0    2023-08-24             1487.41       386594  217131        839.0   
1    2023-08-23             1315.81       337607  192291        841.0   
2    2023-08-22             1335.84       340325  176229        839.0   
3    2023-08-21             2210.01       595105  278400       1284.0   
4    2023-08-20             2148.53       555740  289280       1366.0   
..          ...                 ...          ...     ...          ...   
854  2020-07-29              466.70        64296   48464        460.0   
855  2020-07-28              385.50        59499   43216        386.0   
856  2020-07-27              618.21       112787   81808        770.0   
857  2020-07-26              771.01       145720  109439       1003.0   
858  2020-07-25              770.21       132626  101376        928.0   

    Reporting starts Reporting ends  
0         2020-07-25     2023-08-24  
1       

In [25]:
fb_data["Day"]=pd.to_datetime(fb_data["Day"])

In [26]:
fb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 859 non-null    datetime64[ns]
 1   Amount spent (EUR)  859 non-null    float64       
 2   Impressions         859 non-null    int64         
 3   Reach               859 non-null    int64         
 4   Link clicks         817 non-null    float64       
 5   Reporting starts    859 non-null    object        
 6   Reporting ends      859 non-null    object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 47.1+ KB


In [27]:
fb_data.sort_values(by='Link clicks', ascending=False).tail(50)

,Day,Amount spent (EUR),Impressions,Reach,Link clicks,Reporting starts,Reporting ends
432,2022-06-16,19.76,4954,4830,2.0,2020-07-25,2023-08-24
622,2021-12-04,21.58,3640,2911,2.0,2020-07-25,2023-08-24
808,2020-11-05,1.23,270,266,2.0,2020-07-25,2023-08-24
593,2022-01-02,4.83,4692,4517,2.0,2020-07-25,2023-08-24
635,2021-11-21,9.57,1228,1174,1.0,2020-07-25,2023-08-24
431,2022-06-17,16.29,3629,3426,1.0,2020-07-25,2023-08-24
430,2022-06-18,15.94,4069,3729,1.0,2020-07-25,2023-08-24
429,2022-06-19,16.57,4213,3968,1.0,2020-07-25,2023-08-24
427,2022-06-21,24.76,8091,7248,NaN,2020-07-25,2023-08-24
428,2022-06-20,17.35,5925,5740,NaN,2020-07-25,2023-08-24


In [28]:
fb_data['Impressions'].isnull().sum()

0

In [29]:
fb_data=fb_data.fillna(0)





In [30]:
fb_data['Link clicks'].isnull().sum()

0

In [31]:
fb_data['Link clicks']=fb_data['Link clicks'].astype(int)

In [32]:
fb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 859 non-null    datetime64[ns]
 1   Amount spent (EUR)  859 non-null    float64       
 2   Impressions         859 non-null    int64         
 3   Reach               859 non-null    int64         
 4   Link clicks         859 non-null    int64         
 5   Reporting starts    859 non-null    object        
 6   Reporting ends      859 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 47.1+ KB


In [33]:
fb_data=fb_data.rename(columns={'Amount spent (EUR)':'fb_costs', 'Impressions':'fb_impressions', 'Link clicks':'fb_clicks'})

In [34]:
fb_data

,Day,fb_costs,fb_impressions,Reach,fb_clicks,Reporting starts,Reporting ends
0,2023-08-24,1487.41,386594,217131,839,2020-07-25,2023-08-24
1,2023-08-23,1315.81,337607,192291,841,2020-07-25,2023-08-24
2,2023-08-22,1335.84,340325,176229,839,2020-07-25,2023-08-24
3,2023-08-21,2210.01,595105,278400,1284,2020-07-25,2023-08-24
4,2023-08-20,2148.53,555740,289280,1366,2020-07-25,2023-08-24
...,...,...,...,...,...,...,...
854,2020-07-29,466.70,64296,48464,460,2020-07-25,2023-08-24
855,2020-07-28,385.50,59499,43216,386,2020-07-25,2023-08-24
856,2020-07-27,618.21,112787,81808,770,2020-07-25,2023-08-24
857,2020-07-26,771.01,145720,109439,1003,2020-07-25,2023-08-24


In [35]:
fb_data.drop(columns=['Reach', 'Reporting starts', 'Reporting ends'], inplace=True)

In [36]:
fb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Day             859 non-null    datetime64[ns]
 1   fb_costs        859 non-null    float64       
 2   fb_impressions  859 non-null    int64         
 3   fb_clicks       859 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 27.0 KB


In [37]:
fb_data_cleaned=fb_data

In [38]:
fb_data_cleaned.sort_values(by='Day')

,Day,fb_costs,fb_impressions,fb_clicks
858,2020-07-25,770.21,132626,928
857,2020-07-26,771.01,145720,1003
856,2020-07-27,618.21,112787,770
855,2020-07-28,385.50,59499,386
854,2020-07-29,466.70,64296,460
...,...,...,...,...
4,2023-08-20,2148.53,555740,1366
3,2023-08-21,2210.01,595105,1284
2,2023-08-22,1335.84,340325,839
1,2023-08-23,1315.81,337607,841


## google_data clean up 

In [39]:
google_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1126 entries, 0 to 1125
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Tag           1126 non-null   datetime64[ns]
 1   Währungscode  1126 non-null   object        
 2   Kosten        1126 non-null   object        
 3   Conversions   1126 non-null   object        
 4   Impressionen  1126 non-null   float64       
 5   Klicks        1126 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 52.9+ KB


In [40]:
google_data.sort_values(by='Tag')

,Tag,Währungscode,Kosten,Conversions,Impressionen,Klicks
166,2020-07-25,EUR,"1,53","0,00",67.000,29
150,2020-07-26,EUR,"0,35","0,00",60.000,27
374,2020-07-27,EUR,"4,34","0,00",164.000,75
893,2020-07-28,EUR,"4,32","0,00",107.000,56
359,2020-07-29,EUR,"1,23","0,00",97.000,44
...,...,...,...,...,...,...
731,2023-08-23,EUR,"275,86","34,41",52.861,537
445,2023-08-24,EUR,"299,70","41,97",62.666,554
335,2023-08-25,EUR,"279,64","22,88",37.510,512
406,2023-08-26,EUR,"295,23","17,29",54.014,517


In [41]:
google_data.drop(columns=['Währungscode', 'Conversions'], inplace=True)

In [42]:
google_data["Kosten"] = google_data.loc[:,["Kosten"]].replace({",":"."}, regex=True)

In [43]:
google_data['Kosten']=google_data['Kosten'].astype(float)

In [44]:
google_data['Impressionen']=google_data['Impressionen'].astype(int)

In [45]:
google_data_renamed=google_data.rename(columns={"Tag":"Day", 'Kosten':'google_costs', 'Impressionen':'google_impressions', 'Klicks':'google_clicks'})

In [46]:
google_data_cleaned=google_data_renamed

In [47]:
google_data_cleaned.shape

(1126, 4)

# Join tables

In [48]:
df=pd.merge(shop_data_cleaned,fb_data_cleaned, on="Day", how="left").merge(google_data_cleaned, on="Day", how='left').merge(tt_data_cleaned, on='Day', how='left').fillna(0)
df

,Day,orders,total_sales,fb_costs,fb_impressions,fb_clicks,google_costs,google_impressions,google_clicks,tt_costs,tt_impressions,tt_clicks
0,2021-07-01,10,414.85,257.01,36200.0,129.0,1.17,39.0,20.0,0.0,0.0,0.0
1,2021-07-02,6,229.95,250.30,33040.0,150.0,0.84,14.0,7.0,0.0,0.0,0.0
2,2021-07-03,10,459.89,248.59,33031.0,129.0,0.72,12.0,6.0,0.0,0.0,0.0
3,2021-07-04,17,508.36,258.05,37463.0,119.0,0.07,13.0,5.0,0.0,0.0,0.0
4,2021-07-05,13,399.88,257.47,35963.0,142.0,0.38,34.0,16.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
780,2023-08-20,294,13616.97,2148.53,555740.0,1366.0,466.56,70.0,801.0,0.0,0.0,0.0
781,2023-08-21,356,15348.42,2210.01,595105.0,1284.0,400.58,51.0,782.0,0.0,0.0,0.0
782,2023-08-22,150,6054.53,1335.84,340325.0,839.0,289.46,50.0,619.0,0.0,0.0,0.0
783,2023-08-23,130,4810.67,1315.81,337607.0,841.0,275.86,52.0,537.0,0.0,0.0,0.0


In [49]:
df[df.google_clicks.isnull()]

,Day,orders,total_sales,fb_costs,fb_impressions,fb_clicks,google_costs,google_impressions,google_clicks,tt_costs,tt_impressions,tt_clicks


In [50]:
df[df.fb_costs.isnull()]

,Day,orders,total_sales,fb_costs,fb_impressions,fb_clicks,google_costs,google_impressions,google_clicks,tt_costs,tt_impressions,tt_clicks


In [51]:
df.sort_values(by='Day')

,Day,orders,total_sales,fb_costs,fb_impressions,fb_clicks,google_costs,google_impressions,google_clicks,tt_costs,tt_impressions,tt_clicks
0,2021-07-01,10,414.85,257.01,36200.0,129.0,1.17,39.0,20.0,0.0,0.0,0.0
1,2021-07-02,6,229.95,250.30,33040.0,150.0,0.84,14.0,7.0,0.0,0.0,0.0
2,2021-07-03,10,459.89,248.59,33031.0,129.0,0.72,12.0,6.0,0.0,0.0,0.0
3,2021-07-04,17,508.36,258.05,37463.0,119.0,0.07,13.0,5.0,0.0,0.0,0.0
4,2021-07-05,13,399.88,257.47,35963.0,142.0,0.38,34.0,16.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
780,2023-08-20,294,13616.97,2148.53,555740.0,1366.0,466.56,70.0,801.0,0.0,0.0,0.0
781,2023-08-21,356,15348.42,2210.01,595105.0,1284.0,400.58,51.0,782.0,0.0,0.0,0.0
782,2023-08-22,150,6054.53,1335.84,340325.0,839.0,289.46,50.0,619.0,0.0,0.0,0.0
783,2023-08-23,130,4810.67,1315.81,337607.0,841.0,275.86,52.0,537.0,0.0,0.0,0.0


In [65]:
df[df.orders==0].count()

Day                   22
orders                22
total_sales           22
fb_costs              22
fb_impressions        22
fb_clicks             22
google_costs          22
google_impressions    22
google_clicks         22
tt_costs              22
tt_impressions        22
tt_clicks             22
dtype: int64

In [53]:
df.isnull().sum()

Day                   0
orders                0
total_sales           0
fb_costs              0
fb_impressions        0
fb_clicks             0
google_costs          0
google_impressions    0
google_clicks         0
tt_costs              0
tt_impressions        0
tt_clicks             0
dtype: int64

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 785 non-null    datetime64[ns]
 1   orders              785 non-null    int64         
 2   total_sales         785 non-null    float64       
 3   fb_costs            785 non-null    float64       
 4   fb_impressions      785 non-null    float64       
 5   fb_clicks           785 non-null    float64       
 6   google_costs        785 non-null    float64       
 7   google_impressions  785 non-null    float64       
 8   google_clicks       785 non-null    float64       
 9   tt_costs            785 non-null    float64       
 10  tt_impressions      785 non-null    float64       
 11  tt_clicks           785 non-null    float64       
dtypes: datetime64[ns](1), float64(10), int64(1)
memory usage: 73.7 KB


In [58]:
columns_to_convert = ['fb_impressions', 'fb_clicks', 'google_clicks', 'google_impressions', 'tt_clicks', 'tt_impressions']

# Convert specified columns to integer data type
df[columns_to_convert] = df[columns_to_convert].astype(int)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785 entries, 0 to 784
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Day                 785 non-null    datetime64[ns]
 1   orders              785 non-null    int64         
 2   total_sales         785 non-null    float64       
 3   fb_costs            785 non-null    float64       
 4   fb_impressions      785 non-null    int64         
 5   fb_clicks           785 non-null    int64         
 6   google_costs        785 non-null    float64       
 7   google_impressions  785 non-null    int64         
 8   google_clicks       785 non-null    int64         
 9   tt_costs            785 non-null    float64       
 10  tt_impressions      785 non-null    int64         
 11  tt_clicks           785 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(7)
memory usage: 73.7 KB


In [63]:
df.to_csv('../raw_data/df.csv')

In [62]:
google_data_cleaned.to_csv('../raw_data/google_data_cleaned.csv')

In [61]:
fb_data_cleaned.to_csv('../raw_data/fb_data_cleaned.csv')

In [60]:
tt_data_cleaned.to_csv('../raw_data/tt_data_cleaned.csv')

In [64]:
shop_data_cleaned.to_csv('../raw_data/shop_data_cleaned.csv')